### Import Dependencies

In [6]:
#!pip install stable-baselines3[extra]

SyntaxError: invalid syntax (1062612526.py, line 2)

In [3]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv	# vectorize environment for parallel training
from stable_baselines3.common.evaluation import evaluate_policy
 

### 2. Load Environment

In [5]:
environment_name = 'CartPole-v0'
env = gym.make(environment_name, render_mode='human')

In [3]:
episodes = 10
for episode in range(1, episodes+1):
	state = env.reset()	# reset environment to initial state and get initial observation
	done = False
	score = 0

	while not done:
		env.render()
		action = env.action_space.sample()
		observation, reward, done, truncated, info = env.step(action)
		score += reward
	print(f'Episode: {episode}, Score: {score}')
env.close()

/home/aaron/rl_tutorial/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 1, Score: 27.0
Episode: 2, Score: 16.0
Episode: 3, Score: 23.0
Episode: 4, Score: 12.0
Episode: 5, Score: 23.0
Episode: 6, Score: 19.0
Episode: 7, Score: 75.0
Episode: 8, Score: 16.0
Episode: 9, Score: 38.0
Episode: 10, Score: 11.0


### Understanding the environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [35]:
env.action_space

Discrete(2)

In [36]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

### 3. Train an RL Model

In [9]:
log_path = os.path.join('training', 'logs')	# create the directories first!
log_path

'training/logs'

In [10]:
env = gym.make(environment_name, render_mode='human')
env = DummyVecEnv([lambda: env]) # type: ignore --> wrapper for non-vectorized environment
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)	# defining the "agent" using a default neural network

Using cpu device


In [11]:
model.learn(total_timesteps=20000)

Logging to training/logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 1505 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1047       |
|    iterations           | 2          |
|    time_elapsed         | 3          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00431065 |
|    clip_fraction        | 0.0375     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.55      |
|    explained_variance   | 0.19       |
|    learning_rate        | 0.0003     |
|    loss                 | 9.11       |
|    n_updates            | 110        |
|    policy_gradient_loss | -0.00572   |
|    value_loss           | 48.6       |
----------------------------------------
---------------------

### 4. Save and Reload Model

In [12]:
PPO_path = os.path.join('training', 'saved_models', 'PPO_model_CartPole')

In [13]:
model.save(PPO_path)

In [18]:
del model
model.learn(total_timesteps=20000)

NameError: name 'model' is not defined

In [40]:
model = PPO.load(PPO_path, env=env)

### 5. Evaluation

In [25]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/aaron/rl_tutorial/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/home/aaron/rl_tutorial/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


(500.0, 0.0)

In [29]:
env.close()

### 6. Test Model

In [33]:
action, _ = model.predict(obs)
action

array([1])

In [42]:
env.step(action)

(array([[ 0.0246581 , -0.39061308, -0.02047036,  0.5279979 ]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

In [43]:
episodes = 10
for episode in range(1, episodes+1):
	obs = env.reset()	# reset environment to initial state and get initial observation
	done = False
	score = 0

	while not done:
		env.render()
		action, _ = model.predict(obs)	# now using the model prediction
		obs, reward, done, truncated = env.step(action)
		score += reward
	print(f'Episode: {episode}, Score: {score}')
#env.close()

Episode: 1, Score: [200.]
Episode: 2, Score: [200.]
Episode: 3, Score: [200.]
Episode: 4, Score: [200.]
Episode: 5, Score: [200.]
Episode: 6, Score: [200.]
Episode: 7, Score: [200.]
Episode: 8, Score: [200.]
Episode: 9, Score: [200.]
Episode: 10, Score: [200.]


In [50]:
# reward
env.reset()
action, _ = model.predict(obs)
env.step(action)

/home/aaron/rl_tutorial/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


(array([[ 0.01123324,  0.17523214, -0.0409498 , -0.2795826 ]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

### 7. Viewing Logs in Tensorboard

In [52]:
training_log_path = os.path.join(log_path, 'PPO_2')
training_log_path

'training/logs/PPO_2'

In [ ]:
#!tensorboard --logdir={training_log_path}

### 8. Adding a callback to the training stage

In [22]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [26]:
save_path = os.path.join('training', 'saved_models')

In [37]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env]) # type: ignore --> wrapper for non-vectorized environment
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)


Using cpu device


In [38]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)

In [39]:
model.learn(total_timesteps=50000, callback=eval_callback)

Logging to training/logs/PPO_7
-----------------------------
| time/              |      |
|    fps             | 1586 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1136         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0071692094 |
|    clip_fraction        | 0.0791       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.687       |
|    explained_variance   | 0.00128      |
|    learning_rate        | 0.0003       |
|    loss                 | 10.3         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0118      |
|    value_loss           | 66.5         |
----------------------------

In [40]:
env_rendered = gym.make(environment_name, render_mode='human')
env_rendered = DummyVecEnv([lambda: env_rendered]) # type: ignore --> wrapper for non-vectorized environment

In [41]:
evaluate_policy(model, env_rendered, n_eval_episodes=3, render=True)
env_rendered.close()

### 9. Changing Policies (Network architecture)

In [32]:
new_arch = dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])	# new architecture for the neural network: actor (4x128) and critic (4x128)

In [42]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs=dict(net_arch=new_arch))

Using cpu device


In [43]:
model.learn(total_timesteps=50000, callback=eval_callback)

Logging to training/logs/PPO_8
-----------------------------
| time/              |      |
|    fps             | 1189 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 823         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011589021 |
|    clip_fraction        | 0.149       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.683      |
|    explained_variance   | -0.000365   |
|    learning_rate        | 0.0003      |
|    loss                 | 2.66        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0173     |
|    value_loss           | 17.9        |
-----------------------------------------
---

/home/aaron/rl_tutorial/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 200         |
|    mean_reward          | 200         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.011195697 |
|    clip_fraction        | 0.108       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.572      |
|    explained_variance   | 0.489       |
|    learning_rate        | 0.0003      |
|    loss                 | 16.4        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0168     |
|    value_loss           | 44          |
-----------------------------------------
------------------------------
| time/              |       |
|    fps             | 638   |
|    iterations      | 5     |
|    time_elapsed    | 16    |


### 10. Using an alternative Algorithm

In [45]:
from stable_baselines3 import DQN

In [46]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [47]:
model.learn(total_timesteps=20000)

Logging to training/logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.969    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 5219     |
|    time_elapsed     | 0        |
|    total_timesteps  | 65       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.933    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 5531     |
|    time_elapsed     | 0        |
|    total_timesteps  | 142      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.899    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 5476     |
|    time_elapsed     | 0        |
|    total_timesteps  | 213      |
----------------------------------
------------------------

In [ ]:
#model.save()
#DQN.load()